### SEVER DEFINITIONS


In [1]:
# Necessary Imports
from flask import Flask, jsonify
from werkzeug.routing import IntegerConverter
from _csv import Dialect as _Dialect
import csv
import copy
import os
from rtls_slave import aoa_main, pixel_calculate

# PARAMETER FOR COMPRESSING THE PIXEL POINTS
comp_parameter = .5

# Handle Negative Values
class SignedIntConverter(IntegerConverter):
    regex = r'-?\d+'
app = Flask(__name__)
app.url_map.converters['signed_int'] = SignedIntConverter
#Store input values: AoA and Horizontal Angle

aoabefore = {}
aoa= {}
lev = []

@app.route('/')
def home():
    return "This is the HomePage"

#Routing Settings
@app.route('/send/<signed_int:horiz>', methods=['GET'])
def get_angle(horiz):
    global aoa
    global aoabefore
    print("Plane Angle: " + str(horiz))
    aoa_result = aoa_main(sleep_time=5)
    print(aoa_result)
    input_letter = input("If you want to add this data to list type 'y', otherwise press Enter")
    if input_letter!='y':
        return jsonify({'state':'Try Again'})
    lev.append(horiz)
    aoabefore = copy.deepcopy(aoa)
    aoa = aoa_result
        
    return jsonify({'state': 'success'})

@app.route('/calculate/', methods=['GET'])
def calculate():
    print("Before Turning："+str(aoabefore)+'\n'+"After Turning： " + str(aoa) +'\n'+"Rotation Angle： "+ str(lev))
    #aoa_image()
    u = []
    v = []

    set_aoa = set(aoa)
    set_aoabefore = set(aoabefore)

    for slave in set_aoa.intersection(set_aoabefore):
        minnumber = min(len(aoabefore[slave]), len(aoa[slave]))
        f = open(f"./aoa_results/{slave.replace(':','_')}_before.csv",'a',newline="")
        f.truncate()
        writer = csv.writer(f)
        
        p_count = 0
        for i in range(2,minnumber-2):
            for j in range(2,minnumber-2):
                (temp1, temp2) = pixel_calculate(lev[-2] - lev[-1], aoabefore[slave][i], aoa[slave][j],beta = comp_parameter)

                if (temp1 != 1):
                    u.append(temp1)
                    v.append(temp2)
                    tupx = (temp1,temp2)
                    writer.writerow(tupx)
                
        
        f.close()

    
    return jsonify({'state': 'success'})

### RUNNING THE SEVER

In [2]:
comp_parameter = .6
app.run(host='0.0.0.0',port = 5000, debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.19.217.212:5000
Press CTRL+C to quit


Plane Angle: 86
Master : <RTLSNode(CC26x2 Master, started daemon 7236)> 
Passives : [] 
All : [<RTLSNode(CC26x2 Master, started daemon 7236)>]
Devices Reset
Start scan for 5 sec
Scan Results: [{'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -69, 'advSID': 1, 'periodicAdvInt': 240}, {'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -66, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -59, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -62, 'advSID': 1, 'periodicAdvInt': 240}]
Try to connect to : 80:6F:B0:EE:9C:67
Connected to : 80:6F:B0:EE:9C:67 with connection handle 0
Try to connect to : 80:6F:B0:EE:AA:13
Connected to : 80:6F:B0:EE:AA:13 with connection handle 1
AOA Params Set for Connection Handle: 0
AOA Params Set for Connection Handle: 1
AOA Callback Set for All Connection Handlers
AOA Started with Connection Handle: 0
AOA Started with Connection Handle: 1
Going to sleep for 5 sec
Try to st

10.19.190.195 - - [24/Mar/2023 14:10:11] "GET /send/86 HTTP/1.1" 200 -


Plane Angle: 94
Master : <RTLSNode(CC26x2 Master, started daemon 9424)> 
Passives : [] 
All : [<RTLSNode(CC26x2 Master, started daemon 9424)>]
Devices Reset
Start scan for 5 sec
Scan Results: [{'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -67, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -65, 'advSID': 255, 'periodicAdvInt': 0}, {'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -65, 'advSID': 1, 'periodicAdvInt': 240}, {'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -65, 'advSID': 1, 'periodicAdvInt': 240}]
Try to connect to : 80:6F:B0:EE:9C:67
Connected to : 80:6F:B0:EE:9C:67 with connection handle 0
Try to connect to : 80:6F:B0:EE:AA:13
Connected to : 80:6F:B0:EE:AA:13 with connection handle 1
AOA Params Set for Connection Handle: 0
AOA Params Set for Connection Handle: 1
AOA Callback Set for All Connection Handlers
AOA Started with Connection Handle: 0
AOA Started with Connection Handle: 1
Going to sleep for 5 sec
Try to st

10.19.190.195 - - [24/Mar/2023 14:11:00] "GET /send/94 HTTP/1.1" 200 -


Plane Angle: 89
Master : <RTLSNode(CC26x2 Master, started daemon 10440)> 
Passives : [] 
All : [<RTLSNode(CC26x2 Master, started daemon 10440)>]
Devices Reset
Start scan for 5 sec
Scan Results: [{'addr': '80:6F:B0:EE:9C:67', 'addrType': 0, 'rssi': -68, 'advSID': 1, 'periodicAdvInt': 240}, {'addr': '80:6F:B0:EE:AA:13', 'addrType': 0, 'rssi': -72, 'advSID': 1, 'periodicAdvInt': 240}]
Try to connect to : 80:6F:B0:EE:9C:67
Connected to : 80:6F:B0:EE:9C:67 with connection handle 0
Try to connect to : 80:6F:B0:EE:AA:13
Connected to : 80:6F:B0:EE:AA:13 with connection handle 1
AOA Params Set for Connection Handle: 0
AOA Params Set for Connection Handle: 1
AOA Callback Set for All Connection Handlers
AOA Started with Connection Handle: 0
AOA Started with Connection Handle: 1
Going to sleep for 5 sec
Try to stop AOA result parsing threadSTOP Command Received

AOA Stopped
Master Disconnected
Done
{'80:6F:B0:EE:9C:67': [12, 14, 13, 13, 12, 13, 17, 15, 15, 17, 16], '80:6F:B0:EE:AA:13': [-16, -21, 

10.19.190.195 - - [24/Mar/2023 14:11:53] "GET /send/89 HTTP/1.1" 200 -
f:\intern_VisBLE\VisBLE_INTERN\AoA Point Cloud\rtls_slave.py:261: RuntimeWarning: invalid value encountered in arccos
  azimuth1 = np.arccos(cosAOAangle1/sinelvation1)
f:\intern_VisBLE\VisBLE_INTERN\AoA Point Cloud\rtls_slave.py:258: RuntimeWarning: invalid value encountered in arccos
  elvation1 = np.arccos((cosAOAangle2-coslevelangle*cosAOAangle1)/sinlevelangle)
10.19.190.195 - - [24/Mar/2023 14:11:56] "GET /calculate/ HTTP/1.1" 200 -


Before Turning：{'80:6F:B0:EE:9C:67': [21, 19, 17, 17, 13, 15, 16, 16, 16, 16, 18], '80:6F:B0:EE:AA:13': [-6, -9, -11, -13, -12, -9, -7, -13, -10, -9]}
After Turning： {'80:6F:B0:EE:9C:67': [12, 14, 13, 13, 12, 13, 17, 15, 15, 17, 16], '80:6F:B0:EE:AA:13': [-16, -21, -16, -16, -26, -23, -19, -14, -19, -18, -12]}
Rotation Angle： [86, 89]
